# Load annotated data

In [13]:
import json
import pprint 
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import pandas as pd

# dataset_name = 'meme_retrieval_data'
# dataset_name = 'figmemes'
# dataset_name = 'figmemes_cot'
dataset_name = 'figmemes_cot_few_shot'
# dataset_name = 'memecap'

img_prompt_respond_file = ''
gt_data_file = ''
if dataset_name == 'meme_retrieval_data':
    img_prompt_respond_file = '../data/meme_retrieval_data/filtered_meme_configs_5_attributes_meme_retri.json'
elif dataset_name == 'figmemes':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_figmemes_multi_labels.json'
    gt_data_file = '../data/figmemes/figmemes_annotations.tsv'
elif dataset_name == 'figmemes_cot':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_figmemes_cot.json'
    gt_data_file = '../data/figmemes/cot_samples_gt.tsv'
elif dataset_name == 'figmemes_cot_few_shot':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_figmemes_cot_few_shot.json'
    gt_data_file = '../data/figmemes/cot_samples_gt.tsv'
elif  dataset_name == 'memecap':
    img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'

# load predicted data
meme_configs = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    meme_configs = json.load(json_file)
len(meme_configs)

# load ground truth data
gt_data = pd.read_csv(gt_data_file, sep='\t', index_col='img_name')

In [14]:
gt_data

,allusion,exaggeration,irony,anthrop,metaphor,contrast,year,arts,real,mixed,infograph
img_name,,,,,,,,,,,
1458964859691.jpg,1,0,0,0,0,0,2017,1,0,0,0
1464007930878.jpg,0,0,0,0,0,0,2020,0,1,0,0
1467472694126.png,0,0,1,0,0,0,2017,1,0,0,0
1468935750176.jpg,1,1,0,0,0,0,2019,1,0,0,0
1470534047335.jpg,0,0,0,0,0,0,2017,0,1,0,0
1470839669535.jpg,0,0,1,0,0,0,2017,1,0,0,0
1472045568054.jpg,0,0,0,0,0,0,2018,0,1,0,0
1472180356237.jpg,1,1,0,0,1,1,2017,0,0,1,0
1474915955788.png,0,0,0,0,0,1,2017,0,0,1,0


In [15]:
# gt_data.loc['01144951214.png'][:6].tolist()

In [16]:
def label2vec(label_list: list):
    labels = ['allusion', 'exaggeration', 'irony', 'anthrop', 'metaphor', 'contrast']
    vec = np.zeros((1, len(labels)))
    for label in label_list:
        vec[0, labels.index(label)] = 1
    return vec

label2vec(['exaggeration'])

array([[0., 1., 0., 0., 0., 0.]])

In [17]:
meme_configs[0]

{'image_dir': './data/figmemes/images/1458964859691.jpg',
 'prompt': 'Tasks: \n1. Extract the texts on the meme;\n2. Explain the meme from three perspectives: the humor of the meme; how the meme conveys the humor; And, the emotion behind the meme;\n3. Choose suitable literary devices from the given candidates;\n4. Choose suitable emotional words from the given candidates\n\nThe context of the meme: {}\n\nEmotion labels: anger, fear, surprise, sadness, disgust, contempt, happiness, none\n\nLiterary devices: \n**Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n \n**Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n \n**Irony**: Similar to Sarcasm. Use of words that convey a meaning that is the opposite of its usual meaning/mock someone or something with caustic or bitter use of words.\n  \n**Anthropomorphism**: Similar to Zoomorphi

In [18]:
predictions = []
gt = []
for idx, meme_conf in enumerate(meme_configs):
    predictions.append(label2vec(meme_conf['literary device']))
    gt.append(gt_data.loc[meme_conf['image_dir'].split('/')[-1]][:6].tolist())

predictions = np.array(predictions).squeeze()
gt = np.array(gt)
predictions.shape, gt.shape

((11, 6), (11, 6))

In [21]:
predictions

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0.],
       [0., 1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.]])

In [19]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

print('macro:', f1_score(gt.flatten(), predictions.flatten(), average='macro'))
print('micro:', f1_score(gt.flatten(), predictions.flatten(), average='micro'))
print(classification_report(gt.flatten(), predictions.flatten()))

macro: 0.49302987197724035
micro: 0.5909090909090909
              precision    recall  f1-score   support

           0       0.79      0.65      0.72        52
           1       0.22      0.36      0.27        14

    accuracy                           0.59        66
   macro avg       0.50      0.51      0.49        66
weighted avg       0.67      0.59      0.62        66



In [20]:
def gimme_f1s(y_true, y_pred):
    print('zero')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=0))
    print('one')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=1))
    print()
    print('zero')
    f1s = ['micro', 'macro', 'weighted', 'samples']
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=0, average=score)*100
        print(score)
        print(f1)
        print()
    print('one')
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=1, average=score)*100
        print(score)
        print(f1)
        print()

gimme_f1s(gt, predictions)

zero
              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.25      0.33      0.29         3
           2       0.14      0.50      0.22         2
           3       0.50      1.00      0.67         1
           4       0.00      0.00      0.00         2
           5       0.17      0.50      0.25         2

   micro avg       0.22      0.36      0.27        14
   macro avg       0.22      0.43      0.28        14
weighted avg       0.20      0.36      0.25        14
 samples avg       0.26      0.25      0.23        14

one
              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.25      0.33      0.29         3
           2       0.14      0.50      0.22         2
           3       0.50      1.00      0.67         1
           4       1.00      0.00      0.00         2
           5       0.17      0.50      0.25         2

   micro avg  